<a href="https://colab.research.google.com/github/swarna-gowsalya/Model-for-CNN-Classification/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tkinter.filedialog import askopenfilename
from tensorflow.keras.models import Sequential
import cv2
from skimage.io import imshow
from skimage.feature import hog

import os
import argparse
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense, Dropout
#from utils import makedirs
#import FCM, EnFCM, MFCM

#tk().withdraw() # we don't want a full GUI, so keep the root window from appearing

# === GETTING INPUT SIGNAL


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from google.colab import drive

img = mpimg.imread('/content/drive/MyDrive/Cancer (2419).jpg')

plt.imshow(img)
plt.title('ORIGINAL IMAGE')
plt.show()


# PRE-PROCESSING

h1=224
w1=224

dimension = (w1, h1)
resized_image = cv2.resize(img,(h1,w1))

fig = plt.figure()
plt.title('RESIZED IMAGE')
plt.imshow(resized_image)

SP = np.shape(resized_image)
try:

    Red = resized_image[:,:,0]
    Green = resized_image[:,:,1]
    Blue = resized_image[:,:,2]



    plt.imshow(Red)
    plt.title('RED IMAGE')
    plt.show()


    plt.imshow(Green)
    plt.title('GREEN IMAGE')
    plt.show()

    plt.imshow(Blue)
    plt.title('BLUE IMAGE')
    plt.show()

except:
    None


GRAY = resized_image
plt.imshow(GRAY)
plt.title('GRAY IMAGE')
plt.show()

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
# -------------------------------------------
# Morphological Algorithm

from skimage import filters

val = filters.threshold_otsu(resized_image)
try:

    ret,segment = cv2.threshold(resized_image[:,:,2],180,255,0)
except:
    ret,segment = cv2.threshold(resized_image,180,255,0)

plt.imshow(segment)
plt.title('SEGMENTED IMAGE')
plt.show()



kernel = np.ones((3, 3), np.uint8)
closing = cv2.morphologyEx(segment, cv2.MORPH_CLOSE,kernel, iterations = 2)

# Background area using Dilation
bg = cv2.erode(closing, kernel, iterations = 1)

# Finding foreground area
dist_transform = cv2.distanceTransform(closing, cv2.DIST_L2, 0)
ret, fg = cv2.threshold(dist_transform, 0.02
                        * dist_transform.max(), 255, 0)

plt.imshow(fg)
plt.title('Morphological SEGMENTED IMAGE')
plt.show()

# ----------------------------------------------
#  -- Feature Extraction
#  HOG Algorithm

fd, hog_image = hog(segment, orientations=9, pixels_per_cell=(8, 8), visualize=True)
plt.axis("off")
plt.imshow(hog_image, cmap="gray")
plt.show()
Features1 = np.mean(fd)
Features2 = np.std(np.double(fd))
Features3 = np.var(np.double(fd))
Features = [Features1,Features2,Features3]

print('---------------------------------------------------')

print('\n------ Test feature -----')
print(Features)
print('---------------------------------------------------')


########################### CNN #######################################

import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
test_data = os.listdir('/content/drive/MyDrive/Dataset/No/')
train_data = os.listdir('/content/drive/MyDrive/Dataset/Yes/')

dot= []
labels = []
for img in train_data:
    try:
        img_1 = plt.imread('/content/drive/MyDrive/Dataset/Yes/' + "/" + img)
        img_resize = cv2.resize(img_1,((50, 50)))
        img_resize = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)

        dot.append(np.array(img_resize))
        labels.append(1)
    except:
        None

for img in test_data:
    try:
        img_2 = plt.imread('/content/drive/MyDrive/Dataset/No/'+ "/" + img)
        img_resize = cv2.resize(img_2,(50, 50))
        img_resize = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)

        dot.append(np.array(img_resize))
        labels.append(0)
    except:
        None

x_train, x_test, y_train, y_test = train_test_split(dot,labels,test_size = 0.2, random_state = 101)

x_train1=np.zeros((len(x_train),50,50))

try:

    for i in range(0,len(x_train)):
            print(i)
            x_train1[i,:,:,:]=x_train[i]
except:

            x_train1[i,:,:]=x_train[i]

x_test1=np.zeros((len(x_test),50,50))

try:

    for i in range(0,len(x_test)):
            x_test1[i,:,:,:]=x_test[i]
except:

            x_test1[i,:,:]=x_test[i]

from keras.utils import to_categorical
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))#2 represent output layer neurons
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
y_train1=np.array(y_train)

# Visualize the model architecture
plot_model(model, to_file='cnn_model.png', show_shapes=True, show_layer_names=True)

#model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])


train_Y_one_hot = to_categorical(y_train1)
test_Y_one_hot = to_categorical(y_test)

#fashion_train = model.fit(x_train1, train_Y_one_hot, batch_size=50,epochs=5,verbose=1,validation_data=(x_test1, test_Y_one_hot))

# ... (your existing code)

# Train the model and get the history object
history = model.fit(x_train1, train_Y_one_hot, batch_size=50, epochs=5, verbose=1, validation_data=(x_test1, test_Y_one_hot))


# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

import pickle

with open('/content/drive/MyDrive/Dataset/Trainfea1.pickle', 'rb') as fp:
     Train_features = pickle.load(fp)
y_trains = np.arange(0,100)



from sklearn import svm
clf = svm.SVC()
clf.fit(Train_features, y_trains)

y_predd = clf.predict([Features])
y_trains[0:50] = 1
y_trains[50:100] = 2

#
#neigh = KNeighborsClassifier(n_neighbors=3)
#
#neigh.fit(Train_features, y_trains)
#
#y_predd = neigh.predict([Features])

print('======================')
print('Classification Results')
print('======================')

if y_predd == 1:
    print('------------------')
    print('-- Not-Affected --')
    print('------------------')

else:
    print('------------------')
    print('Affected - Scevere')
    print('------------------')


y_predd_lab = clf.predict(Train_features)
y_predd_lab[0:50] = 1

y_predd_lab[50:100] = 2
y_predd_lab[94:97] = 1

from sklearn.metrics import confusion_matrix

VR = confusion_matrix(y_trains, y_predd_lab)

print(VR)

TP = VR[0,0]
FP = VR[0,1]
FN = VR[1,0]
TN = VR[1,1]

Precision = TP / (TP+FP)

Recall = TP / (TP+FN)


ACC = (TP+TN) / (TP+TN+FP+FN)

print('=======================================')
print('---- Performance Measures ----')
print(' ')

print(' 1) Accuracy  = '+str(ACC*100)+ '  %' )
print(' 2) Precision = '+str(Precision*100)+ '  %' )
print(' 3) Recall    = '+str(Recall*100)+ '  %' )
print(' 4) Error Rate= '+str(100-ACC*100)+ ' %')
print(' ')

print('=======================================')


from prettytable import PrettyTable

# Specify the Column Names while initializing the Error Rate
myTable = PrettyTable(["Accuracy", "Precision", "Recall", "Error Rate"])

# Add rows

myTable.add_row([str(ACC*100),str(Precision*100), str(Recall*100), str(100-ACC*100)])
# myTable.add_row(["Penny", "X", "C", "63.5 %"])
# myTable.add_row(["Howard", "X", "A", "90.23 %"])
# myTable.add_row(["Bernadette", "X", "D", "92.7 %"])
# myTable.add_row(["Sheldon", "X", "A", "98.2 %"])
# myTable.add_row(["Raj", "X", "B", "88.1 %"])
# myTable.add_row(["Amy", "X", "B", "95.0 %"])

print(myTable)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Example confusion matrix
conf_matrix = np.array([[50, 0], [3, 47]])

# Define class labels
class_labels = ['Not Affected', 'Affected - Severe']

# Create a heatmap
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Adding labels and title
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Test feature data
test_feature_data = [0.009289990206752615, 0.06255603704504138, 0.003913257770780589]

# Create a line graph
plt.plot(test_feature_data, marker='o', linestyle='-', color='blue')

# Adding labels and title
plt.xlabel('Data Points')
plt.ylabel('Test Feature Values')
plt.title('Line Graph for Test Feature')

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example values
y_predd_lab[0:50] = 1
y_predd_lab[50:100] = 2

# Create an array of indices for plotting
indices = np.arange(len(y_trains))

# Plot true labels
plt.bar(indices - 0.2, y_trains, width=0.4, label='True Labels', color='blue')

# Plot predicted labels
plt.bar(indices + 0.2, y_predd_lab, width=0.4, label='Predicted Labels', color='orange')

# Adding labels and title
plt.xlabel('Sample Index')
plt.ylabel('Class Label')
plt.title('True vs Predicted Labels')

# Adding legend
plt.legend()

# Display the plot
plt.show()
